In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import analysis.yinf as yinf
import analysis.products as products
from data.util import *
import os, sys, re
ML_DIR = os.path.expandvars("$SCRATCH/mlreco_cell/")
NETS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(ML_DIR))])))
SIM_DIR = os.path.expandvars("$SCRATCH/larsim/")
SIMS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(SIM_DIR)) if "reco" in d])))

def parse_stats_dir(stats_dir = './stats'):
    stats_files = []
    for root, dirs, files in os.walk(stats_dir):
        rel_path = os.path.relpath(root, stats_dir)
        if rel_path.count('/') != 1: continue
        slash_i = rel_path.find('/')
        header = [rel_path[:slash_i], rel_path[slash_i+1:]]
        for f in files:
            epoch, _ = inf_file_info(f)
            thres = re.findall('thres([0-9]*[.]?[0-9]+)', f)
            if len(thres) == 1: 
                thres = float(thres[0])
            else:
                thres = -1
            stats_files.append(header+[epoch, thres, root+'/'+f])
    return pd.DataFrame(stats_files, columns=['net', 'inf', 'epoch', 'thres', 'file'])

STATS = parse_stats_dir().sort_values(by=['epoch'])

def select_stats(net=None, inf=None, thres=None, first=0, last=-1, stride=1, stats_df=STATS):
    if net is not None:
        stats_df = stats_df[stats_df.net==net]
    if inf is not None:
        stats_df = stats_df[stats_df.inf==inf]
    if thres is not None:
        stats_df = stats_df[stats_df.thres==thres]
    if last == -1:
        return stats_df.sort_values(by=['epoch']).iloc[first::stride, :]['file']
    return stats_df.sort_values(by=['epoch']).iloc[first:last+1:stride, :]['file']


In [2]:
NETS

{0: '.ipynb_checkpoints',
 1: 'random_sparse',
 2: 'singleElectronHit_ghost2D_1652-rep4',
 3: 'singleElectronHit_ghost2D_1652-rep4_L1',
 4: 'singleElectronHit_ghost2D_1652-rep4_L1-2',
 5: 'singleElectronHit_ghost2D_1652-rep4_L10',
 6: 'singleElectronHit_ghost3D-164',
 7: 'singleElectronHit_ghost3D-164_L1-1',
 8: 'singleElectronHit_ghost3D-164_L1-3',
 9: 'singleElectronWire-30k_ghost3D-164',
 10: 'singleElectronWire_ghost3D-164',
 11: 'singleElectronWire_ghost3D-164_L1-3',
 12: 'singleElectronWire_ghost3D-164_L10',
 13: 'singleElectronWire_ghost3D-164_WeightedCE',
 14: 'singleElectronWire_ghost3D-164_WeightedCE-1',
 15: 'singleElectronWire_ghost3D-164_WeightedCE2',
 16: 'sparse',
 17: 'sparse_singleElectronHit_ghost2D_1652',
 18: 'sparse_singleElectronHit_ghost3D',
 19: 'sparse_singleMu',
 20: 'sparse_singleMuHit_ghost2D_841',
 21: 'sparse_singleMuHit_ghost2D_853',
 22: 'sparse_singleMuHit_ghost3D',
 23: 'sparse_singleMu_ghost',
 24: 'sparse_singleMu_ghost3D'}

In [3]:
SIMS

{0: 'reco_1GeV_ElectronHit_xy',
 1: 'reco_1GeV_ElectronWire_xy',
 2: 'reco_1GeV_Electron_parsed',
 3: 'reco_1GeV_Electron_xy',
 4: 'reco_1GeV_MuHit_xy',
 5: 'reco_1GeV_Pion_parsed',
 6: 'reco_1GeV_Pion_xy',
 7: 'reco_1GeV_parsed',
 8: 'reco_1GeV_parsed_test',
 9: 'reco_1GeV_test',
 10: 'reco_1GeV_xy',
 11: 'reco_1GeV_xy_test'}

In [4]:
def plot_train_summary(net_dir, start_epoch = 2):
    print(net_dir)
    summary = pd.read_csv(ML_DIR+net_dir+'/summaries_0.csv', delim_whitespace=True)
    summary = summary.drop(columns=["epoch"])
    summary[["train_loss", "valid_loss"]][start_epoch:].plot()
    summary[["train_acc", "valid_acc"]][start_epoch:].plot()
    summary[["train_time", "valid_time"]].plot()
    
def compute_yinf_stats(net_dir, xy_dir, start_epoch = 0, stride=1, n_yinf_files = 10, n_events=50, true_thres=0, inf_dir="inference"):
    stats_dir = "./stats/"+net_dir+'/'+inf_dir
    os.makedirs(stats_dir, exist_ok=True)
    print(net_dir)
    yinf_dir = ML_DIR+net_dir+'/'+inf_dir
    yinf_epoch = files_info(os.listdir(yinf_dir), [0], inf_file_info)
    n_epochs = len(yinf_epoch.keys())
    for epoch in range(start_epoch, n_epochs, stride):
        yinf.yinf_avg_stats(n_yinf_files, n_events, epoch, true_thres, stats_dir, False,  SIM_DIR+xy_dir, yinf_dir)

def find_n_last(text, pattern, n):
    if n==1: return text.rfind(pattern)
    return find_n_last(text[:text.rfind(pattern)], pattern, n-1)

def plot_yinf_stats(*stats_files):
    stats_files = np.concatenate(stats_files)
    labels = [f[find_n_last(f, '/', 3)+1:f.rfind('_')] for f in stats_files]
    yinf.compare_yinf_stats(plt.figure(), stats_files, labels)

def plot_yinf(figs, net_dir, xy_dir, epoch = -1, file_i=0, event = 0, downsample=(1,1,1), thres=None, true_thres=0, plot_slice=False, plot_lims=None, view_angle=None, inf_dir="inference"):
    fig, fig_hist, fig_inf, fig_hist_inf, fig_true, fig_true_hist, fig_true_inf, fig_true_hist_inf= figs
    yinf_dir = ML_DIR+net_dir+'/'+inf_dir
    yinf_epoch = files_info(os.listdir(yinf_dir), [0], inf_file_info)
    epoch_i = sorted(yinf_epoch.keys())[epoch]
    yinf_file = yinf_epoch[epoch_i][file_i]
    plot_lims, thres = yinf.plot_yinf(fig, fig_hist, fig_inf, fig_hist_inf, 
                                      event, downsample, thres, true_thres, 
                                      plot_slice, plot_lims, view_angle, SIM_DIR+xy_dir, yinf_dir+'/'+yinf_file)
    products.compare_true_active(fig_true, fig_true_hist,  fig_true_inf, fig_true_hist_inf, 
                                 event, downsample, true_thres, 
                                 plot_slice, plot_lims, view_angle, False, SIM_DIR+xy_dir+'/'+inf_file_info(yinf_file)[1].replace("yinf", "xy")+'.npz')
    x_lim, y_lim, z_lim = plot_lims
    return x_lim, y_lim, z_lim, thres

class PlotYinfWidget():   
    def __init__(self, out, **kwargs):
        if kwargs.get("event") is None:
            kwargs["event"] = 0
        self.view_angle = kwargs["view_angle"]
        kwargs["plot_slice"] = False
        self.figs = [plt.figure() for _ in range(8)]
        kwargs["figs"] = self.figs
        self.Size = self.figs[0].get_size_inches() 
        for fig in self.figs:
            fig.set_size_inches(self.Size[0]/3, self.Size[1]/3, forward=True)
        self.kwargs = kwargs
        self.out = out
    def get_widgets(self):
        bnext = widgets.Button(description='Next')
        bprev = widgets.Button(description='Previous')
        bset = widgets.Button(description='Set')
        breset = widgets.Button(description='Reset')
        bnext.on_click(callback.next)
        bprev.on_click(callback.prev)
        bset.on_click(callback.set)
        breset.on_click(callback.reset)
        self.xmin = widgets.IntSlider(
             step=1,
             description='xmin')
        self.xmax = widgets.IntSlider(
             step=1,
             description='xmax')
        self.ymin = widgets.IntSlider(
             step=1,
             description='ymin')
        self.ymax = widgets.IntSlider(
             step=1,
             description='ymax')
        self.zmin = widgets.IntSlider(
             step=1,
             description='zmin')
        self.zmax = widgets.IntSlider(
             step=1,
             description='zmax')
        self.thres = widgets.FloatSlider(
             min=0,
             max=1.0,
             step=0.01,
             description='threshold')
        self.elev = widgets.FloatSlider(
             value=self.view_angle[0],
             min=-90,
             max=90,
             step=1,
             description='elevation')
        self.azim = widgets.FloatSlider(
             value=self.view_angle[1],
             min=-90,
             max=90,
             step=1,
             description='azimuth')
        self.slice = widgets.Checkbox(
            value=False,
            description='2D slice')
        self.plots = [widgets.Checkbox(
            value=True,
            description='Fig %d'%(i+self.figs[0].number)) for i in range(8)]
        self._plot_event()
        return bnext, bprev, bset, breset, \
            self.xmin, self.xmax, self.ymin, self.ymax, self.zmin, self.zmax, \
            self.thres, self.elev, self.azim, self.slice, self.plots
            
    def next(self, event):
        self.kwargs["event"] += 1
        self._clear_figs()
        self._clear_vals()
        self._plot_event() 
    def prev(self, event):
        if self.kwargs["event"]>0:
            self.kwargs["event"] -= 1
        self._clear_figs()
        self._clear_vals()
        self._plot_event()
    def set(self, event):
        self._clear_figs()
        self._read_vals()
        self._plot_event()
    def reset(self, event):
        self._clear_figs()
        self._clear_vals(True)
        self._plot_event()
    
    def _plot_event(self):
        if not self.slice.value:
            self.xmin.max=self.xmax.max=10000
            self.xmin.min=self.xmax.min=-10000
        self.ymin.max=self.ymax.max=self.zmin.max=self.zmax.max=10000
        self.ymin.min=self.ymax.min=self.zmin.min=self.zmax.min=-10000
        with self.out:
            clear_output()
            defaults = plot_yinf(**self.kwargs)
            if self.slice.value:
                self.xmax.value = defaults[0][1]
                self.xmin.value = defaults[0][0]
            else:
                self.xmin.max=self.xmax.max=self.xmax.value =defaults[0][1]
                self.xmin.min=self.xmax.min=self.xmin.value =defaults[0][0]
            self.ymin.max=self.ymax.max=self.ymax.value = defaults[1][1]
            self.ymin.min=self.ymax.min=self.ymin.value = defaults[1][0]
            self.zmin.max=self.zmax.max=self.zmax.value = defaults[2][1]
            self.zmin.min=self.zmax.min=self.zmin.value = defaults[2][0]
            self.thres.value = defaults[3]
            self.kwargs["plot_lims"] = defaults[:3]
            self.kwargs["thres"] = defaults[3]

    def _read_vals(self):
        self.kwargs["plot_slice"] = self.slice.value
        if self.slice.value and self.kwargs["plot_lims"][0][0] != self.xmin.value:
            self.xmax.value = self.xmin.value+1
            self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value), None, None)
        else:
            self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value),
                                        (self.ymin.value, self.ymax.value),
                                        (self.zmin.value, self.zmax.value))
        self.kwargs["thres"] = self.thres.value
        self.kwargs["view_angle"] = (self.elev.value, self.azim.value)
        self.kwargs["figs"] = [fig if plot.value else None for fig, plot in zip(self.figs, self.plots)]
        for fig, plot in zip(self.figs, self.plots):
            if plot.value:
                fig.set_size_inches(self.Size[0]*2, self.Size[1]*3, forward=True)
    def _clear_vals(self, reset=False):
        if reset and self.slice.value:
            self.kwargs["plot_lims"] = ((self.xmin.value, self.xmax.value), None, None)
        else:
            self.slice.value = self.kwargs["plot_slice"] = False
            self.kwargs["plot_lims"] = None
        self.kwargs["thres"] = None
        self.elev.value, self.azim.value = self.kwargs["view_angle"] = self.view_angle
    def _clear_figs(self):
        for fig in self.figs:
            fig.clf()



In [ ]:
net = 'sparse_singleElectronHit_ghost2D_1652'
#plot_train_summary(net)
compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir="inference2")
#compute_yinf_stats(net, SIMS[0], 1, true_thres=0, n_yinf_files=5)
plot_yinf_stats(select_stats(net, "inference", 0.5, -2, stride = 2),  select_stats(net, "inference2"))

In [ ]:
net = 'singleElectronHit_ghost2D_1652-rep4'
plot_train_summary(net)
compute_yinf_stats(net, SIMS[0], 1, true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')
compute_yinf_stats(net, SIMS[0], 1, true_thres=0, n_yinf_files=5, inf_dir='inf_train')
plot_yinf_stats(select_stats(net, "inference", 0.5, stride = 2),  select_stats(net, "inf_train"))


In [ ]:
net = 'singleElectronHit_ghost2D_1652-rep4_L1-2'
plot_train_summary(net, 5)
#compute_yinf_stats(net, SIMS[0], 1, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[0], 1, true_thres=0, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')

plot_yinf_stats(select_stats(net, "inf_train", 0.5))
plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 2))
plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 2), select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, stride = 4), 
                select_stats(net, "inference", 0, stride = 4), 
                select_stats(net, "inf_train"))

In [ ]:
net = 'singleElectronHit_ghost3D-164'
#plot_train_summary(net, 5)
#compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')
#compute_yinf_stats(net, SIMS[0], 1, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[0], 1, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 2))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 2), select_stats(net, "inf_train"))

out = widgets.Output()
callback = PlotYinfWidget(out, net_dir=net, xy_dir=SIMS[0], true_thres=0.5, view_angle=(0,90), inf_dir='inference')
bnext, bprev, bset, breset, xmin, xmax, ymin, ymax, zmin, zmax, thres, elev, azim, slic, plots = callback.get_widgets()
bs = widgets.HBox([bprev, bnext, bset, breset])
plots_inf = widgets.HBox(plots[:4])
plots_active = widgets.HBox(plots[4:])
viewangle = widgets.HBox([elev, azim])
x_lims = widgets.HBox([xmin, xmax, slic])
y_lims = widgets.HBox([ymin, ymax])
z_lims = widgets.HBox([zmin, zmax])
widgets.VBox([bs, plots_inf, plots_active, thres, viewangle, x_lims, y_lims, z_lims, out])


In [ ]:
net = 'singleElectronHit_ghost3D-164_L1-1'
plot_train_summary(net, 5)
#compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')
compute_yinf_stats(net, SIMS[0], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[0], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5,  -5, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, stride = 1), select_stats(net, "inf_train"))

In [ ]:
net = 'singleElectronHit_ghost3D-164_L1-3'
plot_train_summary(net, 8)
#compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')
#compute_yinf_stats(net, SIMS[0], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[0], 1, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, -10, stride = 2))
#plot_yinf_stats(select_stats(net, "inference", 0.5, stride = 1), select_stats(net, "inf_train"))

In [ ]:
net = 'singleElectronWire_ghost3D-164'
plot_train_summary(net, 20)
#compute_yinf_stats(net, SIMS[4], true_thres=0.5, n_yinf_files=5, inf_dir='inf_muhit')
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -6, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -1), select_stats(net, "inf_train"), select_stats(net, "inf_muhit"))

out = widgets.Output()
callback = PlotYinfWidget(out, net_dir=net, xy_dir=SIMS[1], true_thres=0.5, view_angle=(0,90), inf_dir='inference')
bnext, bprev, bset, breset, xmin, xmax, ymin, ymax, zmin, zmax, thres, elev, azim, slic, plots = callback.get_widgets()
bs = widgets.HBox([bprev, bnext, bset, breset])
plots_inf = widgets.HBox(plots[:4])
plots_active = widgets.HBox(plots[4:])
viewangle = widgets.HBox([elev, azim])
x_lims = widgets.HBox([xmin, xmax, slic])
y_lims = widgets.HBox([ymin, ymax])
z_lims = widgets.HBox([zmin, zmax])
widgets.VBox([bs, plots_inf, plots_active, thres, viewangle, x_lims, y_lims, z_lims, out])

In [ ]:
net = 'singleElectronWire_ghost3D-164_WeightedCE'
#plot_train_summary(net, 0)
#compute_yinf_stats(net, SIMS[4], true_thres=0.5, n_yinf_files=5, inf_dir='inf_muhit')
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -6, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -1), select_stats(net, "inf_train"), select_stats(net, "inf_muhit"))
out = widgets.Output()
callback = PlotYinfWidget(out, net_dir=net, xy_dir=SIMS[1], true_thres=0.5, view_angle=(0,90), inf_dir='inference')
bnext, bprev, bset, breset, xmin, xmax, ymin, ymax, zmin, zmax, thres, elev, azim, slic, plots = callback.get_widgets()
bs = widgets.HBox([bprev, bnext, bset, breset])
plots_inf = widgets.HBox(plots[:4])
plots_active = widgets.HBox(plots[4:])
viewangle = widgets.HBox([elev, azim])
x_lims = widgets.HBox([xmin, xmax, slic])
y_lims = widgets.HBox([ymin, ymax])
z_lims = widgets.HBox([zmin, zmax])
widgets.VBox([bs, plots_inf, plots_active, thres, viewangle, x_lims, y_lims, z_lims, out])

In [ ]:
net = 'singleElectronWire_ghost3D-164_WeightedCE2'
plot_train_summary(net, 0)
#compute_yinf_stats(net, SIMS[4], true_thres=0.5, n_yinf_files=5, inf_dir='inf_muhit')
compute_yinf_stats(net, SIMS[1], 5, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, -6, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -1), select_stats(net, "inf_train"), select_stats(net, "inf_muhit"))


In [ ]:
net = 'singleElectronWire_ghost3D-164_WeightedCE-1'
plot_train_summary(net, 0)
#compute_yinf_stats(net, SIMS[4], true_thres=0.5, n_yinf_files=5, inf_dir='inf_muhit')
compute_yinf_stats(net, SIMS[1], 10, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, -6, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -1), select_stats(net, "inf_train"), select_stats(net, "inf_muhit"))
out = widgets.Output()
callback = PlotYinfWidget(out, net_dir=net, xy_dir=SIMS[1], true_thres=0.5, view_angle=(0,90), inf_dir='inference')
bnext, bprev, bset, breset, xmin, xmax, ymin, ymax, zmin, zmax, thres, elev, azim, slic, plots = callback.get_widgets()
bs = widgets.HBox([bprev, bnext, bset, breset])
plots_inf = widgets.HBox(plots[:4])
plots_active = widgets.HBox(plots[4:])
viewangle = widgets.HBox([elev, azim])
x_lims = widgets.HBox([xmin, xmax, slic])
y_lims = widgets.HBox([ymin, ymax])
z_lims = widgets.HBox([zmin, zmax])
widgets.VBox([bs, plots_inf, plots_active, thres, viewangle, x_lims, y_lims, z_lims, out])


In [ ]:
net = 'singleElectronWire-30k_ghost3D-164'
plot_train_summary(net, 20)
#compute_yinf_stats(net, SIMS[4], true_thres=0.5, n_yinf_files=5, inf_dir='inf_muhit')
compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, -6, stride = 1))
plot_yinf_stats(select_stats(net, "inference", 0.5, -1), select_stats(net, "inf_train"), select_stats(net, "inf_muhit"))

out = widgets.Output()
callback = PlotYinfWidget(out, net_dir=net, xy_dir=SIMS[1], true_thres=0.5, view_angle=(0,90), inf_dir='inference')
bnext, bprev, bset, breset, xmin, xmax, ymin, ymax, zmin, zmax, thres, elev, azim, slic, plots = callback.get_widgets()
bs = widgets.HBox([bprev, bnext, bset, breset])
plots_inf = widgets.HBox(plots[:4])
plots_active = widgets.HBox(plots[4:])
viewangle = widgets.HBox([elev, azim])
x_lims = widgets.HBox([xmin, xmax, slic])
y_lims = widgets.HBox([ymin, ymax])
z_lims = widgets.HBox([zmin, zmax])
widgets.VBox([bs, plots_inf, plots_active, thres, viewangle, x_lims, y_lims, z_lims, out])


In [ ]:
net = 'singleElectronWire_ghost3D-164_L10'
plot_train_summary(net, 10)
#compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')
compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 2), select_stats(net, "inf_train"))

In [ ]:
net = 'singleElectronWire_ghost3D-164_L1-3'
plot_train_summary(net, 20)
#compute_yinf_stats(net, SIMS[0], true_thres=0.5, n_yinf_files=5, inf_dir='inf_train')
compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0.5, n_yinf_files=5)
#compute_yinf_stats(net, SIMS[1], 1, stride=2, true_thres=0, n_yinf_files=5)

#plot_yinf_stats(select_stats(net, "inf_train"))
plot_yinf_stats(select_stats(net, "inference", 0.5, -6, stride = 1))
#plot_yinf_stats(select_stats(net, "inference", 0.5, -12, stride = 2), select_stats(net, "inf_train"))

In [ ]:
THRES=0.5

plot_yinf_stats(select_stats('sparse_singleElectronHit_ghost3D', 'inference', first=-1), 
               select_stats('singleElectronHit_ghost2D_1652-rep4_L1-2', 'inference', THRES, first= -1),
                select_stats('singleElectronHit_ghost2D_1652-rep4', 'inference', THRES,first= -1),
               select_stats('sparse_singleElectronHit_ghost2D_1652', 'inference', THRES, first= -1),
               select_stats('singleElectronHit_ghost3D-164', 'inference', THRES, -1),
                select_stats('singleElectronHit_ghost3D-164_L1-1', 'inference', THRES, first= -1),
               select_stats('singleElectronHit_ghost3D-164_L1-3', 'inference', THRES, first= -1),
                select_stats('singleElectronWire_ghost3D-164', 'inference', THRES, -1),)
                #['./stats//sparse_singleElectronHit_ghost2D_1652/inference/epoch27_stats-N250.npz',
                    #'./stats//singleElectronHit_ghost2D_1652-rep4/inference/epoch20_stats-N250.npz', 
                   #'./stats//singleElectronHit_ghost3D-164/inference/epoch18_stats-N250.npz',])
            
plot_yinf_stats(
               select_stats('singleElectronHit_ghost2D_1652-rep4_L1-2', 'inference', THRES, first= -1),
                select_stats('singleElectronHit_ghost2D_1652-rep4', 'inference', THRES,first= -1),
               select_stats('sparse_singleElectronHit_ghost2D_1652', 'inference', THRES, first= -1),)

plot_yinf_stats(select_stats('sparse_singleElectronHit_ghost3D', 'inference', first=-1), 
    select_stats('singleElectronHit_ghost3D-164', 'inference', THRES, -1),
                select_stats('singleElectronHit_ghost3D-164_L1-1', 'inference', THRES, first= -1),
               select_stats('singleElectronHit_ghost3D-164_L1-3', 'inference', THRES, first= -1),
                select_stats('singleElectronWire_ghost3D-164', 'inference', THRES, -1),
               select_stats('singleElectronWire_ghost3D-164_L1-3', 'inference', THRES, -1),)
            
plot_yinf_stats(select_stats('singleElectronHit_ghost3D-164', 'inference', THRES, -1),
               select_stats('singleElectronHit_ghost3D-164', 'inf_train',  THRES, first=-1),
               select_stats('singleElectronHit_ghost2D_1652-rep4', 'inference', THRES, -1),
               select_stats('singleElectronHit_ghost2D_1652-rep4', 'inf_train',  THRES, first=-1),
                select_stats('singleElectronHit_ghost2D_1652-rep4_L1-2', 'inference', THRES, -1),
                select_stats('singleElectronHit_ghost2D_1652-rep4_L1-2', 'inf_train',  THRES, first= -1),
                select_stats('sparse_singleElectronHit_ghost2D_1652', 'inference', first= -1),
                select_stats('singleElectronHit_ghost3D-164_L1-1', 'inference', THRES, first= -1)
            )


In [ ]:
THRES=0.5
plot_yinf_stats(select_stats('sparse_singleElectronHit_ghost3D', 'inference', first=-1), 
    select_stats('singleElectronHit_ghost3D-164', 'inference', THRES, -1),
                #select_stats('singleElectronHit_ghost3D-164_L1-1', 'inference', THRES, first= -1),
               select_stats('singleElectronHit_ghost3D-164_L1-3', 'inference', THRES, first= -1),
                select_stats('singleElectronWire_ghost3D-164', 'inference', THRES, -1),
                select_stats('singleElectronWire-30k_ghost3D-164', 'inference', THRES, -1),
               select_stats('singleElectronWire_ghost3D-164_L1-3', 'inference', THRES, -1),
               select_stats('singleElectronWire_ghost3D-164_L10', 'inference', THRES, -1),)